In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPool2D, BatchNormalization
from keras.utils import to_categorical
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

train_data = train_df.drop(['label'], axis = 1)
train_target = train_df['label']

train_data = train_data.values.reshape(-1,28,28,1)
test_data = test_df.values.reshape(-1,28,28,1)

train_target = to_categorical(train_target)

In [3]:
model = Sequential()
model.add(Conv2D(64, kernel_size = 3, activation = 'relu', padding = 'same', input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size = 3, activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = 2))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size = 3, activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size = 3, activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = 2))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size = 3, activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size = 3, activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = 2))
model.add(Dropout(0.25))

model.add(Conv2D(512, kernel_size = 3, activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(Conv2D(512, kernel_size = 3, activation = 'relu', padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size = 2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation = 'softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.6, 
                                            min_lr=0.00001)

In [5]:
model.fit(train_data, train_target, validation_split = 0.15, epochs = 40, callbacks = [learning_rate_reduction] )

Instructions for updating:
Use tf.cast instead.
Train on 35700 samples, validate on 6300 samples
Epoch 1/40
35700/35700 [==============================] - 24s 680us/step - loss: 0.2367 - acc: 0.9318 - val_loss: 0.0842 - val_acc: 0.9802
Epoch 2/40
35700/35700 [==============================] - 20s 560us/step - loss: 0.0793 - acc: 0.9792 - val_loss: 0.0510 - val_acc: 0.9876
Epoch 3/40
35700/35700 [==============================] - 20s 574us/step - loss: 0.0617 - acc: 0.9849 - val_loss: 0.0433 - val_acc: 0.9908
Epoch 4/40
35700/35700 [==============================] - 20s 554us/step - loss: 0.0552 - acc: 0.9868 - val_loss: 0.0709 - val_acc: 0.9849
Epoch 5/40
35700/35700 [==============================] - 20s 554us/step - loss: 0.0430 - acc: 0.9898 - val_loss: 0.0476 - val_acc: 0.9897
Epoch 6/40
35700/35700 [==============================] - 20s 552us/step - loss: 0.0398 - acc: 0.9910 - val_loss: 0.0404 - val_acc: 0.9924
Epoch 7/40
35700/35700 [==============================] - 20s 566us/s

In [6]:
results = model.predict(test_data)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("digit_recognizer_submission_3.csv",index=False)